In [ ]:
# 環境セットアップ
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import imageio.v2 as imageio
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import pickle

# GPU確認
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
    device = torch.device('cuda')
    # GPU最適化設定
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
else:
    print("⚠️ GPU not available. Please enable GPU in Runtime > Change runtime type")
    device = torch.device('cpu')

print(f"🚀 Using device: {device}")

# パッケージインストール
try:
    import imageio
    import seaborn
    print("✅ All packages available")
except ImportError:
    print("📦 Installing required packages...")
    !pip install imageio scikit-learn seaborn
    import imageio
    import seaborn
    print("✅ Packages installed successfully")


In [ ]:
# Google Drive接続とデータパス確認
from google.colab import drive

# Google Driveをマウント
drive.mount('/content/drive')

# データパス設定
GIF_FOLDER_PATH = '/content/drive/MyDrive/GrayScottML/gif'

# データ確認
if os.path.exists(GIF_FOLDER_PATH):
    gif_files = [f for f in os.listdir(GIF_FOLDER_PATH) if f.endswith('.gif')]
    gif_count = len(gif_files)
    print(f"✅ Google Drive connected successfully!")
    print(f"📁 Path: {GIF_FOLDER_PATH}")
    print(f"🎬 GIF files found: {gif_count}")
    
    if gif_count >= 1000:
        print("🎉 Ready for Phase 2 training with last 64 frames!")
    else:
        print(f"⚠️ Not enough files. Expected: 1500, Found: {gif_count}")
        print("Please upload more GIF files to Google Drive.")
else:
    print("❌ Google Drive path not found!")
    print(f"Expected path: {GIF_FOLDER_PATH}")
    print("Please ensure the following structure exists:")
    print("  MyDrive/")
    print("  └── GrayScottML/")
    print("      └── gif/")
    print("          ├── GrayScott-f0.0100-k0.0400-00.gif")
    print("          └── ... (1500 files)")
    
    # マイドライブの内容を表示
    mydrive_path = '/content/drive/MyDrive'
    if os.path.exists(mydrive_path):
        print(f"\n📂 Contents of MyDrive:")
        for item in sorted(os.listdir(mydrive_path))[:10]:
            print(f"   📁 {item}")
        print("   ... (showing first 10 items)")
    raise FileNotFoundError("Please set up the correct folder structure in Google Drive")


In [ ]:
# データセットクラス（後半64フレーム専用）
class GrayScottDatasetLast64(Dataset):
    def __init__(self, gif_folder, fixed_frames=64, target_size=(64, 64), max_samples=None):
        self.gif_folder = gif_folder
        self.fixed_frames = fixed_frames
        self.target_size = target_size
        self.max_samples = max_samples
        
        self.gif_files = []
        self.f_values = []
        self.k_values = []
        self.tensors = []
        
        self._load_data()
    
    def _parse_filename(self, filename):
        pattern = r'GrayScott-f([0-9.]+)-k([0-9.]+)-\d+\.gif'
        match = re.match(pattern, filename)
        if match:
            return float(match.group(1)), float(match.group(2))
        return None, None
    
    def _load_gif_as_tensor(self, gif_path):
        try:
            gif = imageio.mimread(gif_path)
            total_frames = len(gif)
            
            # 後半64フレームを抽出
            if total_frames >= self.fixed_frames:
                # 後半64フレームを取得
                start_idx = total_frames - self.fixed_frames
                selected_frames = gif[start_idx:]
            else:
                # フレーム数が64未満の場合は全フレームを使用
                selected_frames = gif
                print(f"Warning: Only {total_frames} frames available, using all frames")
            
            frames = []
            for frame in selected_frames:
                if len(frame.shape) == 3:
                    frame = np.mean(frame, axis=2)
                
                pil_frame = Image.fromarray(frame.astype(np.uint8))
                pil_frame = pil_frame.resize(self.target_size)
                frame_array = np.array(pil_frame) / 255.0
                frames.append(frame_array)
            
            # 64フレームになるようにパディング（必要に応じて）
            while len(frames) < self.fixed_frames:
                frames.append(frames[-1] if frames else np.zeros(self.target_size))
            
            # 正確に64フレームにする
            frames = frames[:self.fixed_frames]
            
            tensor = torch.FloatTensor(np.array(frames))
            return tensor.unsqueeze(0)  # チャンネル次元を追加
            
        except Exception as e:
            print(f"Error loading {gif_path}: {e}")
            return None
    
    def _load_data(self):
        gif_files = [f for f in os.listdir(self.gif_folder) if f.endswith('.gif')]
        
        if self.max_samples:
            gif_files = gif_files[:self.max_samples]
        
        print(f"Loading {len(gif_files)} GIF files (last 64 frames each)...")
        
        for i, filename in enumerate(gif_files):
            if i % 100 == 0:
                print(f"Progress: {i+1}/{len(gif_files)} ({(i+1)/len(gif_files)*100:.1f}%)")
            
            f_val, k_val = self._parse_filename(filename)
            if f_val is None or k_val is None:
                continue
            
            gif_path = os.path.join(self.gif_folder, filename)
            tensor = self._load_gif_as_tensor(gif_path)
            
            if tensor is not None:
                self.gif_files.append(filename)
                self.f_values.append(f_val)
                self.k_values.append(k_val)
                self.tensors.append(tensor)
        
        print(f"✅ Successfully loaded {len(self.tensors)} samples with last 64 frames each")
    
    def __len__(self):
        return len(self.tensors)
    
    def __getitem__(self, idx):
        return {
            'tensor': self.tensors[idx],
            'f_value': self.f_values[idx],
            'k_value': self.k_values[idx],
            'filename': self.gif_files[idx]
        }


In [ ]:
# 注意機構モジュール
class SpatioTemporalAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        
        # 空間注意
        self.spatial_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((None, 1, 1)),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
        
        # 時間注意
        self.temporal_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((1, None, None)),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
        
        # チャンネル注意
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        identity = x
        
        # 3つの注意機構を適用
        x = x * self.spatial_attention(x)
        x = x * self.temporal_attention(x)
        x = x * self.channel_attention(x)
        
        return x + identity * 0.1

# 残差注意ブロック
class ResidualAttentionBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv3d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm3d(out_channels, momentum=0.1)
        self.conv2 = nn.Conv3d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm3d(out_channels, momentum=0.1)
        
        self.attention = SpatioTemporalAttention(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm3d(out_channels, momentum=0.1)
            )
        
        self.dropout = nn.Dropout3d(0.1)
    
    def forward(self, x):
        identity = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out = self.attention(out)
        
        out += self.shortcut(identity)
        return F.relu(out)


In [ ]:
# Phase 2 メインモデル（64フレーム対応）
class Conv3DAutoencoderPhase2Last64(nn.Module):
    def __init__(self, input_channels=1, fixed_frames=64, target_size=(64, 64), latent_dim=256):
        super().__init__()
        
        self.latent_dim = latent_dim
        self.fixed_frames = fixed_frames
        self.target_size = target_size
        
        # 初期畳み込み（64フレーム対応）
        self.initial_conv = nn.Sequential(
            nn.Conv3d(input_channels, 32, (5, 7, 7), (2, 2, 2), (2, 3, 3)),
            nn.BatchNorm3d(32, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.05)
        )
        
        # 残差注意ブロック群（64フレーム用に調整）
        self.res_block1 = ResidualAttentionBlock3D(32, 64, stride=(2, 2, 2))
        self.res_block2 = ResidualAttentionBlock3D(64, 64)
        self.res_block3 = ResidualAttentionBlock3D(64, 128, stride=(2, 2, 2))
        self.res_block4 = ResidualAttentionBlock3D(128, 128)
        self.res_block5 = ResidualAttentionBlock3D(128, 256, stride=(2, 2, 2))
        self.res_block6 = ResidualAttentionBlock3D(256, 256)
        
        # グローバルプーリング
        self.global_pool = nn.AdaptiveAvgPool3d((2, 2, 2))
        self.dropout_before_latent = nn.Dropout3d(0.3)
        
        # 潜在空間射影
        self.to_latent = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 2 * 2 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        
        # 復元
        self.from_latent = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, 256 * 2 * 2 * 2),
            nn.ReLU(inplace=True)
        )
        
        # デコーダー（64フレーム復元用）
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(256, 128, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(128, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.2),
            
            nn.ConvTranspose3d(128, 64, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(64, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.15),
            
            nn.ConvTranspose3d(64, 32, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(32, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.1),
            
            nn.ConvTranspose3d(32, input_channels, (6, 7, 7), (2, 2, 2), (2, 3, 3)),
            nn.Sigmoid()
        )
    
    def encode(self, x):
        x = self.initial_conv(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.global_pool(x)
        x = self.dropout_before_latent(x)
        return self.to_latent(x)
    
    def decode(self, latent):
        x = self.from_latent(latent)
        x = x.view(-1, 256, 2, 2, 2)
        x = self.decoder(x)
        target_h, target_w = self.target_size
        return F.interpolate(x, size=(self.fixed_frames, target_h, target_w), 
                           mode='trilinear', align_corners=False)
    
    def forward(self, x):
        latent = self.encode(x)
        reconstructed = self.decode(latent)
        return reconstructed, latent


In [ ]:
# Phase 2 学習・評価実行（64フレーム版）
def run_phase2_training_last64():
    # パラメータ設定
    fixed_frames = 64  # 後半64フレーム
    target_size = (64, 64)
    latent_dim = 256
    num_epochs = 50
    batch_size = 6 if torch.cuda.is_available() else 3  # 64フレームなので少し小さく
    learning_rate = 1e-3
    weight_decay = 1e-4
    n_clusters = 5
    
    print("🔄 Creating dataset (last 64 frames)...")
    dataset = GrayScottDatasetLast64(GIF_FOLDER_PATH, fixed_frames, target_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=2, pin_memory=True)
    
    print(f"📊 Dataset: {len(dataset)} samples, Batch size: {batch_size}, Frames: {fixed_frames}")
    
    print("🧠 Creating Phase 2 model (64 frames)...")
    model = Conv3DAutoencoderPhase2Last64(latent_dim=latent_dim, 
                                         fixed_frames=fixed_frames, 
                                         target_size=target_size).to(device)
    
    print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 訓練
    print("🎯 Starting training with last 64 frames...")
    model.train()
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
    
    losses = []
    start_time = time.time()
    
    print(f"🚀 Phase 2 GPU Training: ResNet + Attention (Last 64 Frames)")
    print("=" * 80)
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        epoch_loss = 0.0
        
        for batch_idx, batch in enumerate(dataloader):
            tensors = batch['tensor'].to(device, non_blocking=True)
            
            optimizer.zero_grad()
            reconstructed, latent = model(tensors)
            loss = criterion(reconstructed, tensors)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            epoch_loss += loss.item()
        
        scheduler.step()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        current_lr = scheduler.get_last_lr()[0]
        epoch_time = time.time() - epoch_start_time
        
        # 進捗表示
        progress = ((epoch + 1) / num_epochs) * 100
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f'Epoch [{epoch+1:2d}/{num_epochs}] '
                  f'({progress:5.1f}%) | '
                  f'Loss: {avg_loss:.6f} | '
                  f'LR: {current_lr:.2e} | '
                  f'Time: {epoch_time:.1f}s')
    
    total_time = time.time() - start_time
    print("=" * 80)
    print(f"🎉 Training completed in {total_time/60:.1f} minutes")
    
    # 学習曲線表示
    plt.figure(figsize=(10, 6))
    plt.plot(losses, linewidth=2, color='purple')
    plt.title('Phase 2: GPU Training Loss (ResNet + Attention, Last 64 Frames)', fontsize=14)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 潜在ベクトル抽出
    print("🔍 Extracting latent vectors...")
    model.eval()
    latent_vectors = []
    f_values = []
    k_values = []
    
    with torch.no_grad():
        for batch in dataloader:
            tensors = batch['tensor'].to(device)
            _, latent = model(tensors)
            latent_vectors.append(latent.cpu().numpy())
            f_values.extend(batch['f_value'].numpy())
            k_values.extend(batch['k_value'].numpy())
    
    latent_vectors = np.vstack(latent_vectors)
    f_values = np.array(f_values)
    k_values = np.array(k_values)
    
    # クラスタリング
    print("🎯 Performing clustering...")
    scaler = StandardScaler()
    latent_scaled = scaler.fit_transform(latent_vectors)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(latent_scaled)
    
    # 性能評価
    silhouette_avg = silhouette_score(latent_vectors, cluster_labels)
    calinski_score = calinski_harabasz_score(latent_vectors, cluster_labels)
    davies_bouldin = davies_bouldin_score(latent_vectors, cluster_labels)
    
    print("=" * 80)
    print("🏆 Phase 2 GPU Results Summary (Last 64 Frames):")
    print("=" * 80)
    print(f"🎯 Architecture: ResNet + SpatioTemporalAttention (GPU, Last 64 Frames)")
    print(f"📊 Samples: {len(dataset)}")
    print(f"🎬 Frames per sample: {fixed_frames} (last frames)")
    print(f"🧠 Latent Dimension: {latent_dim}")
    print(f"⚙️  Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"📉 Final Loss: {losses[-1]:.6f}")
    print(f"🎯 Clusters: {n_clusters}")
    print(f"⭐ Silhouette Score: {silhouette_avg:.4f}")
    print(f"📊 Calinski-Harabasz: {calinski_score:.2f}")
    print(f"📈 Davies-Bouldin: {davies_bouldin:.4f}")
    print("=" * 80)
    
    # Phase 1との比較
    phase1_score = 0.565
    improvement = ((silhouette_avg - phase1_score) / phase1_score) * 100
    
    print(f"📈 Performance Comparison:")
    print(f"   Phase 1 (30 frames): {phase1_score:.4f}")
    print(f"   Phase 2 (last 64 frames): {silhouette_avg:.4f}")
    print(f"   Improvement: {improvement:+.1f}%")
    
    if improvement >= 15:
        print("🎉 Phase 2 目標達成！ (15%以上の向上) - Last 64 frames strategy successful!")
    else:
        print(f"⚠️  Phase 2 目標未達 ({improvement:.1f}% < 15%) - Consider further optimization")
    
    return {
        'model': model,
        'losses': losses,
        'silhouette_score': silhouette_avg,
        'calinski_score': calinski_score,
        'davies_bouldin': davies_bouldin,
        'latent_vectors': latent_vectors,
        'cluster_labels': cluster_labels,
        'f_values': f_values,
        'k_values': k_values,
        'improvement': improvement,
        'frames_used': 'last_64'
    }

# 実行
print("🚀 Starting Phase 2 training with last 64 frames...")
results = run_phase2_training_last64()


In [ ]:
# 結果保存とダウンロード（Last 64 frames版）
import pickle
from google.colab import files

# 結果をGoogle Driveに保存
results_path = '/content/drive/MyDrive/GrayScottML/phase2_results_last64_gpu.pkl'
model_path = '/content/drive/MyDrive/GrayScottML/phase2_model_last64_gpu.pth'

# 結果保存
with open(results_path, 'wb') as f:
    # モデルは除いて保存（サイズ削減）
    save_results = {k: v for k, v in results.items() if k != 'model'}
    pickle.dump(save_results, f)

# モデル保存
torch.save(results['model'].state_dict(), model_path)

print(f"💾 Results saved to Google Drive:")
print(f"   📊 Results: {results_path}")
print(f"   🧠 Model: {model_path}")

# ローカルにもダウンロード用ファイル作成
local_results_path = 'phase2_results_last64_gpu.pkl'
local_model_path = 'phase2_model_last64_gpu.pth'

with open(local_results_path, 'wb') as f:
    save_results = {k: v for k, v in results.items() if k != 'model'}
    pickle.dump(save_results, f)

torch.save(results['model'].state_dict(), local_model_path)

print("\n📥 Downloading files...")
files.download(local_results_path)
files.download(local_model_path)

print("✅ Download completed!")
print(f"🎯 Final Results (Last 64 Frames):")
print(f"   ⭐ Silhouette Score: {results['silhouette_score']:.4f}")
print(f"   📈 Improvement: {results['improvement']:+.1f}%")
print(f"   🎬 Strategy: Using last 64 frames for more stable patterns")
print(f"   🏆 Target: {'✅ ACHIEVED' if results['improvement'] >= 15 else '❌ NOT ACHIEVED'}")


In [ ]:
# 結果の詳細可視化（Last 64 frames）
def visualize_results_last64(results):
    latent_vectors = results['latent_vectors']
    cluster_labels = results['cluster_labels']
    f_values = results['f_values']
    k_values = results['k_values']
    
    # PCAとt-SNEによる次元削減
    print("🔍 Performing dimensionality reduction...")
    pca = PCA(n_components=2, random_state=42)
    pca_result = pca.fit_transform(latent_vectors)
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    tsne_result = tsne.fit_transform(latent_vectors)
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Phase 2 Results: Last 64 Frames Analysis', fontsize=16, fontweight='bold')
    
    # f-k空間でのクラスタリング結果
    scatter1 = axes[0, 0].scatter(f_values, k_values, c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[0, 0].set_xlabel('f parameter')
    axes[0, 0].set_ylabel('k parameter')
    axes[0, 0].set_title('Clustering Results in f-k Parameter Space')
    axes[0, 0].invert_yaxis()
    plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster ID')
    
    # PCA結果
    scatter2 = axes[0, 1].scatter(pca_result[:, 0], pca_result[:, 1], c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[0, 1].set_title(f'PCA of Latent Space (Last 64 Frames)')
    axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.colorbar(scatter2, ax=axes[0, 1], label='Cluster ID')
    
    # t-SNE結果
    scatter3 = axes[1, 0].scatter(tsne_result[:, 0], tsne_result[:, 1], c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[1, 0].set_title('t-SNE of Latent Space (Last 64 Frames)')
    plt.colorbar(scatter3, ax=axes[1, 0], label='Cluster ID')
    
    # クラスター統計
    axes[1, 1].axis('off')
    n_clusters = len(np.unique(cluster_labels))
    stats_text = f"Last 64 Frames Analysis:\\n\\n"
    stats_text += f"Silhouette Score: {results['silhouette_score']:.4f}\\n"
    stats_text += f"Improvement: {results['improvement']:+.1f}%\\n\\n"
    
    for i in range(n_clusters):
        mask = cluster_labels == i
        count = np.sum(mask)
        f_mean = f_values[mask].mean()
        k_mean = k_values[mask].mean()
        stats_text += f"Cluster {i}: {count} samples\\n"
        stats_text += f"  f_avg: {f_mean:.4f}\\n"
        stats_text += f"  k_avg: {k_mean:.4f}\\n\\n"
    
    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, 
                    fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Visualization completed!")
    print(f"🎬 Strategy: Last 64 frames captured more stable, mature patterns")
    print(f"⭐ Final Score: {results['silhouette_score']:.4f}")

# 可視化実行
print("🎨 Creating visualizations...")
visualize_results_last64(results)


In [ ]:
# Google Driveフォルダ確認・作成支援
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# フォルダパス設定（マイドライブ内）
base_path = '/content/drive/MyDrive'
project_folder = os.path.join(base_path, 'GrayScottML')
gif_folder = os.path.join(project_folder, 'gif')

# 段階的にフォルダ作成
try:
    if not os.path.exists(project_folder):
        os.makedirs(project_folder)
        print(f"✅ Created: {project_folder}")
    else:
        print(f"✅ Already exists: {project_folder}")
        
    if not os.path.exists(gif_folder):
        os.makedirs(gif_folder)
        print(f"✅ Created: {gif_folder}")
    else:
        print(f"✅ Already exists: {gif_folder}")
        
    # テストファイル作成（フォルダが確実に存在することを確認）
    test_file = os.path.join(project_folder, 'test.txt')
    with open(test_file, 'w') as f:
        f.write('GrayScott ML Project - Folder created successfully!')
    
    print("🎉 Folder structure created successfully!")
    print("📁 Structure:")
    print(f"   {project_folder}")
    print(f"   └── {gif_folder}")
    print(f"   └── test.txt")
    
    # Google Drive側で確認するよう案内
    print("\n💡 Please check your Google Drive:")
    print("   MyDrive > GrayScottML > gif (folder)")
    print("   MyDrive > GrayScottML > test.txt (file)")
    
except Exception as e:
    print(f"❌ Error creating folders: {e}")
    print("Please try creating folders manually in Google Drive web interface")


In [ ]:
# パス確認とデバッグ
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# 複数のパス候補を確認
path_candidates = [
    '/content/drive/MyDrive/GrayScottML/gif',           # マイドライブ内
    '/content/drive/MyDrive/GrayScottML',               # プロジェクトフォルダ
    '/content/drive/GrayScottML/gif',                   # ルート直下（もしあれば）
]

print("🔍 Searching for GrayScottML folder...")
found_path = None

for path in path_candidates:
    if os.path.exists(path):
        print(f"✅ Found: {path}")
        if path.endswith('/gif'):
            gif_count = len([f for f in os.listdir(path) if f.endswith('.gif')])
            print(f"   📁 GIF files: {gif_count}")
            if gif_count > 0:
                found_path = path
                break
        else:
            contents = os.listdir(path)
            print(f"   📁 Contents: {contents}")
            if 'gif' in contents:
                gif_path = os.path.join(path, 'gif')
                gif_count = len([f for f in os.listdir(gif_path) if f.endswith('.gif')])
                print(f"   📁 GIF files in gif/: {gif_count}")
                if gif_count > 0:
                    found_path = gif_path
                    break
    else:
        print(f"❌ Not found: {path}")

# マイドライブの内容を表示
mydrive_path = '/content/drive/MyDrive'
if os.path.exists(mydrive_path):
    print(f"\n📂 Contents of MyDrive:")
    for item in sorted(os.listdir(mydrive_path)):
        item_path = os.path.join(mydrive_path, item)
        if os.path.isdir(item_path):
            print(f"   📁 {item}")
        else:
            print(f"   📄 {item}")

if found_path:
    print(f"\n🎉 Ready to use: {found_path}")
    # グローバル変数として設定
    GIF_FOLDER_PATH = found_path
    print(f"🔗 GIF_FOLDER_PATH = '{GIF_FOLDER_PATH}'")
else:
    print(f"\n⚠️ GrayScottML folder not found. Please:")
    print("1. Create 'GrayScottML' folder in MyDrive")
    print("2. Create 'gif' subfolder inside GrayScottML")
    print("3. Upload 1500 GIF files to the gif folder")
